In [1]:
# DIRECTORY SET
import os
import sys
import traceback
from pathlib import Path
base_dir=Path(os.getcwd()).parent.parent.parent
os.chdir(base_dir)
print(os.getcwd())

# ENVIRONMENT VARIABLES
import dotenv
dotenv.load_dotenv()

# DJANGO SETUP
import django
sys.path.append(os.path.abspath(''))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "serverproject.settings")
django.setup()

# Import async modules
import asyncio
from asgiref.sync import sync_to_async

# Import display modules
from IPython.display import display, Markdown

# Import other modules
import faiss

# import reloading
from importlib import reload

# Enable autoreload
%load_ext autoreload
%autoreload 2

# Import custom modules
from destinyapp.models import StreamRecapData

from core import services
from core import utils
from core import controller

d:\DestinyFolder\DestinyRecaps\DestinyRecapsApi\serverproject


In [2]:
video_id="OqVH_MTBQ6k"

In [62]:
stream_recap_data=await utils.get_recap_data(video_id)

annotated_results, major_topics, minor_topics, cost = await services.stream_plot.generate_data(stream_recap_data)

Cost:  0.007353
Cost:  0.00139975
Cost:  0.00227225
Cost:  0.002322
Cost:  0.00236175
Cost:  0.0025397500000000003
Cost:  0.0024720000000000002
Cost:  0.002572
Cost:  0.003057
Cost:  0.0029372499999999998
Cost:  0.0032025
Cost:  0.00305025
Cost:  0.00166125
Cost:  0.0022725
Cost:  0.00248225
Results:  20
Results:  20
Results:  20
Results:  0 Text Chunks:  20  Returning blank equal to text chunks length
Results:  20
Results:  20
Results:  13 Text Chunks:  20  Returning blank equal to text chunks length
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  12


In [63]:
plot_object, annotated_results, plot_segments, category_locations = await services.stream_plot.process_data(stream_recap_data,  annotated_results, major_topics, minor_topics, video_id)

plot_object, cost=await services.stream_plot.annotate_extra(video_id, stream_recap_data, plot_object)      

Counters:  [101, 69]


# save the plot_object to the database
from dataclasses import dataclass, asdict, is_dataclass

stream_recap_data=await utils.get_recap_data(video_id)
stream_recap_data.plot_object=asdict(plot_object)
await sync_to_async(stream_recap_data.save)()

In [5]:
# save plot_object as json to D:\DestinyFolder\DestinyRecaps\DestinyRecapsWeb\frontend\src\apps\stream_plot\plot_data.json
from dataclasses import dataclass, asdict, is_dataclass
from typing import Any
import json
plot_object_json=json.dumps(asdict(plot_object))
with open(r"D:\DestinyFolder\DestinyRecaps\DestinyRecapsWeb\frontend\src\apps\stream_plot\plot_data.json", "w") as f:
    f.write(plot_object_json)

# Prompting

In [6]:
from core.services.stream_plot import extra_annotations as ea

In [7]:
i=11
response=await ea.recap_segment(plot_object.segments[i].category, plot_object.segments[i].texts)
print("Category: ", plot_object.segments[i].category)

Category:  Kamala Harris as Potential Democratic Nominee


In [8]:
print(response)

("Discussion centers on Kamala Harris as a potential Democratic nominee against Trump. Logistically, she could inherit campaign funds, but ethically, skipping over her as vice president seems wrong. The conversation questions if Biden's age is the sole factor influencing this decision and whether stepping down would benefit Harris politically.", 0.00012869999999999998)


In [35]:
i_test=100
print(plot_object.segments[i_test].href)
print(plot_object.segments[i_test].start_time)

https://youtu.be/OqVH_MTBQ6k?t=15188
15173


In [ ]:
services.stream_plot.extra_annotations.annotate_start()

In [27]:
context_offset=500

tasks=[]
for i in range(len(plot_object.segments)):
    task_text=None
    if plot_object.segments[i].texts!=[]:
        start_index=stream_recap_data.transcript.find(plot_object.segments[i].texts[0])
        contextualized_start_index=start_index-context_offset
        if (contextualized_start_index)<0:
            contextualized_start_index=0
        task_text=stream_recap_data.transcript[contextualized_start_index:start_index+len(plot_object.segments[i].texts[0])]
    tasks.append(services.stream_plot.extra_annotations.annotate_start(stream_recap_data, plot_object.segments[i].category, contextualized_start_index, task_text))
start_counts_and_costs=await asyncio.gather(*tasks)

In [26]:
plot_object.segments[5].category

"Biden's Decision Not to Seek Re-election"

In [38]:
for i in range(len(plot_object.segments)):
    if plot_object.segments[i].category=="Biden's Decision Not to Seek Re-election":
        print(i)

5
57
64
69
71
76
78
83


In [41]:
stream_recap_data.transcript.find(plot_object.segments[69].texts[0])-500

161000

In [40]:
plot_object.segments[69].texts

["\n\nC: Eris is 59, though. I mean, so she's a little old.\n\nB: Yeah.\n\nC: You know. God, yeah. I do want to call out any centrist who was like, I'm not going to name any names, of course, but any centrist who was like, Biden, I can't do that. Now you have an opportunity to stick by your guns and condemn Donald Trump and to join the other ticket and to forcefully say, I support the democratic candidate. I think that would be a great idea. And I think everyone who was kind of on the fence, or maybe even they said, I prefer Trump just because of how old Biden is. Maybe now is the time to stick with your guns and your politics, what you say are your politics, and endorse the democratic nominee.\n\nDestiny: Yeah.\n\nC: What do you think about that?\n\nDestiny: True, I asked, oh, I hit that question at the end of my, I did an interview with Tommy or Tom Bill. Y you the impact theory guy? Yeah, I asked him that at the very end. I wanted to get this question. I asked him before. I think it

In [39]:
plot_object.segments[69].href

'https://youtu.be/OqVH_MTBQ6k?t=10906'

In [46]:
len(task_text)

1500

In [61]:
t_index=60
start_index=stream_recap_data.transcript.find(plot_object.segments[t_index].texts[0])
task_text=stream_recap_data.transcript[contextualized_start_index:start_index+len(plot_object.segments[t_index].texts[0])]
real_start_character_index, cost, response=await services.stream_plot.extra_annotations.annotate_start(stream_recap_data, plot_object.segments[t_index].category, stream_recap_data.transcript.find(plot_object.segments[t_index].texts[0])-500, task_text)

ValueError: not enough values to unpack (expected 3, got 2)

In [59]:
print(real_start_character_index)
print(response)

235500
o moderate. Why do you accept dogship moderators? Myron has actually been probably one of the best moderators that I think ive ve had for any debate. I think most of the debates I've had where Myron is moderating, I think he heavily agrees with the other side, like in the rolo TMMSi one and the one with the Suli Mon guy or whatever. But I think he've just done a pretty fair job moderating at all of them. I don't think he's ever done anything wacky or stupid or ever. Yeah. Wait. Okay. I just wanted to read this real quick. I'm sorry. Because this up has to do with a judicial review. I guess I'm going to do anything else with Dan in about an hour. Just so you know. I'll be doing that. So I'm going to have to take off soon to go set up. But let's read this real quick first. Federalist number 78, the judiciary department. This would have been written in. These were all written in, what, 1770? What? Six through eight, I think. Oh, wait, 1780s. Oh, I'm sorry. I might have been. This o

In [60]:
start_counts_and_costs[60]


(131500, 0.00035130000000000003)